In [47]:
import os
from tfx.components import ImportExampleGen
from tfx.components import ExampleValidator
from tfx.components import Pusher
#from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Evaluator
from tfx.components import Transform
from tfx.components import FileBasedExampleGen
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2, trainer_pb2, example_gen_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.extensions.google_cloud_big_query.example_gen.component import BigQueryExampleGen
import tensorflow_data_validation as tfdv

## Paths and InteractiveContext

In [48]:
# Set up pipeline paths
data_root = os.path.join(os.getcwd(), 'felidae_tfrecords_split')
pipeline_root = os.path.join(os.getcwd(), 'tfx_pipeline_output')

# Create Interactive Context
context = InteractiveContext(pipeline_root=pipeline_root)

## ExampleGen

In [49]:
# Ingest data using ImportExampleGen
input_config = example_gen_pb2.Input(splits=[
    example_gen_pb2.Input.Split(name='train', pattern= os.path.join('*', 'train', '*')),
    example_gen_pb2.Input.Split(name='eval', pattern=os.path.join('*', 'eval', '*'))
])

example_gen = ImportExampleGen(input_base=data_root, input_config=input_config)
context.run(example_gen)
display(example_gen)

ImportExampleGen(spec: <tfx.types.standard_component_specs.FileBasedExampleGenSpec object at 0x000001FDABCD1190>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x000001FDABCD11F0>, driver_class: <class 'tfx.components.example_gen.driver.FileBasedDriver'>, component_id: ImportExampleGen, inputs: {}, outputs: {'examples': OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={})})

## StatisticsGen

In [50]:
# Generate data statistics and schema
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)



ExecutionResult(
    component_id: StatisticsGen
    execution_id: 17
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [51]:
#Display the results

display(statistics_gen)
context.show(statistics_gen.outputs['statistics'])

StatisticsGen(spec: <tfx.types.standard_component_specs.StatisticsGenSpec object at 0x000001FDABCD1AC0>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x000001FDABCD1B50>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: StatisticsGen, inputs: {'examples': OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={})}, outputs: {'statistics': OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={})})

## SchemaGen

In [52]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True
)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 18
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [53]:
from tfx.utils import io_utils
from tensorflow_metadata.proto.v0 import schema_pb2

def print_schema(schema_path):
    schema = schema_pb2.Schema()
    io_utils.parse_pbtxt_file(file_name=schema_path, message=schema)
    print(schema)

print_schema(schema_gen.outputs['schema']._artifacts[0].uri + '/schema.pbtxt')


feature {
  name: "image_raw"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "label"
  type: BYTES
  domain: "label"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
string_domain {
  name: "label"
  value: "Cheetah"
  value: "Leopard"
  value: "Lion"
  value: "Puma"
  value: "Tiger"
}



In [54]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'image_raw',BYTES,required,,-
'label',STRING,required,,'label'


,Values
Domain,
'label',"'Cheetah', 'Leopard', 'Lion', 'Puma', 'Tiger'"


### Curating the schema.

In [55]:
schema = tfdv.load_schema_text(schema_gen.outputs['schema']._artifacts[0].uri + '/schema.pbtxt')

In [56]:
# Declare the path to the updated schema directory
UPDATED_SCHEMA_DIR = os.path.join(pipeline_root, 'updated_schema')

# Create the said directory
!mkdir -p {UPDATED_SCHEMA_DIR}

# Declare the path to the schema file
schema_file = os.path.join(UPDATED_SCHEMA_DIR, 'schema.pbtxt')

A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file c:\Users\Guillermo\Desktop\My_projects\TFX_test_pipeline\tfx_pipeline_output\updated_schema already exists.
Error occurred while processing: c:\Users\Guillermo\Desktop\My_projects\TFX_test_pipeline\tfx_pipeline_output\updated_schema.


In [57]:
# Update the schema to include the transformed features
from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow_data_validation.utils import schema_util


# Constants
IMAGE_SIZE = 224  # You can change this according to your model's input size
NUM_CHANNELS = 3


tfdv.set_domain(schema, "label", schema_pb2.IntDomain(name="label", min= 0, max= 4, is_categorical = True))


# Update the schema to include the transformed features
tfdv.get_feature(schema, "label").type = schema_pb2.INT





### Creating a serving enviroment in the schema

In [58]:
# grader-required-cell
schema.default_environment.append('TRAINING')

### START CODE HERE ###
# Hint: Create another default schema environment with name SERVING (pass in a string)
schema.default_environment.append('SERVING')

# Remove Cover_Type feature from SERVING using TFDV
# Hint: Pass in the strings with the name of the feature and environment 
tfdv.get_feature(schema, 'label').not_in_environment.append('SERVING')

In [59]:
def display_serving_schema(schema):
    print('Features in SERVING environment:')
    for feature in schema.feature:
        if 'SERVING' not in feature.not_in_environment:
            print(f"{feature.name} ({feature.type})")

In [60]:
try:
    # Get the schema uri
    schema_uri = schema_gen.outputs['schema']._artifacts[0].uri
    
# for grading since context.run() does not work outside the notebook
except IndexError:
    print("context.run() was no-op")
    schema_path = './pipeline/SchemaGen/schema'
    dir_id = os.listdir(schema_path)[0]
    schema_uri = f'{schema_path}/{dir_id}'

In [61]:
display_serving_schema(schema)

Features in SERVING environment:
image_raw (1)


In [62]:
# Save the curated schema to the said file
tfdv.write_schema_text(schema, schema_file)

Lets check that the schema is the new one. 

In [63]:
# Load the schema from the directory we just created
new_schema = tfdv.load_schema_text(schema_file)

# Display the schema. Check that the Domain column still contains the ranges.
tfdv.display_schema(schema=new_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'image_raw',BYTES,required,,-
'label',INT,required,,min: 0; max: 4


,Values
Domain,
'label',"'Cheetah', 'Leopard', 'Lion', 'Puma', 'Tiger'"


In [64]:
display_serving_schema(new_schema)

Features in SERVING environment:
image_raw (1)


In [65]:
new_schema.default_environment

['TRAINING', 'SERVING']

## Example Validator

In [66]:
# Validate data using ExampleValidator
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 19
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

### Creation of cats_transform.py

In [77]:
%%writefile cats_transform.py

import tensorflow as tf
import tensorflow_transform as tft

# Constants
IMAGE_SIZE = 224  # You can change this according to your model's input size
NUM_CHANNELS = 3

# Feature names
LABEL_KEY = 'label'
IMAGE_KEY = 'image_raw'

# Output feature names after transformation
TRANSFORMED_LABEL_KEY = 'transformed_label'
TRANSFORMED_IMAGE_KEY = 'transformed_image'

def preprocess_image(image_bytes, target_size=(IMAGE_SIZE, IMAGE_SIZE)):

    print(image_bytes)
    image = tf.image.decode_jpeg(image_bytes, channels=NUM_CHANNELS)
    print(tf.shape(image))
    image = tf.image.resize(image, target_size, method=tf.image.ResizeMethod.BICUBIC)
    image = tf.cast(image, tf.float32) / 255.0
    return image

def preprocessing_fn(inputs):
    """Preprocess input features into transformed features."""
    outputs = {}

    print(inputs)

    image_raw = inputs[IMAGE_KEY]

    print(tf.shape(image_raw))

    print(type(image_raw))

    def process_images(image_bytes):
        return preprocess_image(image_bytes)

    image_normalized = tf.map_fn(process_images, image_raw, dtype=tf.float32)

    label = inputs[LABEL_KEY]
    label_integer = tf.strings.to_number(label, out_type=tf.int32)

    label_float = tf.cast(label_integer, tf.float32)

    outputs[TRANSFORMED_IMAGE_KEY] = image_normalized
    outputs[TRANSFORMED_LABEL_KEY] = label_float

    return outputs





Overwriting cats_transform.py


In [78]:
# Train the model using Trainer
#module_file = 'cats_trainer.py'
transform_module_file = 'cats_transform.py'

# Add Transform component
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=transform_module_file
)
context.run(transform)

{'image_raw': <tf.Tensor 'inputs_copy:0' shape=(None, 1) dtype=string>, 'label': <tf.Tensor 'inputs_1_copy:0' shape=(None, 1) dtype=string>}
Tensor("Shape:0", shape=(2,), dtype=int32)
<class 'tensorflow.python.framework.ops.Tensor'>
Tensor("map/while/TensorArrayV2Read/TensorListGetItem:0", shape=(1,), dtype=string)


ValueError: Shape must be rank 0 but is rank 1 for '{{node map/while/DecodeJpeg}} = DecodeJpeg[acceptable_fraction=1, channels=3, dct_method="", fancy_upscaling=true, ratio=1, try_recover_truncated=false](map/while/TensorArrayV2Read/TensorListGetItem)' with input shapes: [1].

In [30]:
%%writefile cats_trainer.py

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tfx.components.trainer.fn_args_utils import FnArgs
from tfx.v1.dsl.components.base import executor_spec
from tfx.v1.dsl.components.trainer import executor as trainer_executor
from tfx.v1.extensions.google_cloud_big_query.example_gen import executor as big_query_executor

# Constants
NUM_CLASSES = 5
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 10000
TRANSFORMED_LABEL_KEY = 'transformed_label'
TRANSFORMED_IMAGE_KEY = 'transformed_image'


def build_model():
    """Create a simple CNN model."""
    inputs = tf.keras.Input(shape=(64, 64, 3))
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(NUM_CLASSES, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model


def input_fn(file_pattern, transformed_metadata, batch_size):
    """Generates features and labels for training or evaluation."""
    def _gzip_reader_fn(filenames):
        """Small utility returning a record reader that can read gzip'ed files."""
        return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

    dataset = tf.data.experimental.CsvDataset(
        file_pattern,
        transformed_metadata.schema.as_feature_spec(),
        batch_size=batch_size,
        reader=_gzip_reader_fn,
        field_delim=',',
        use_quote_delim=False,
        na_value='',
        header=True,
        field_dtype=None,
        use_display_data=False,
        strip_quotes=True,
        skip_blank_lines=False,
        delimiter='\t',
        quotechar='"',
        encoding='utf-8',
        buffer_size=None,
        header_lines=None,
        na_filter=True,
        skiprows=None,
        nrows=None,
        compression='infer',
        thousands=None,
        decimal=b'.',
        lineterminator=None,
        quotechar='"',
        quoting=0,
        doublequote=True,
        escapechar=None,
        comment=None,
        encoding=None,
        dialect=None,
        error_bad_lines=True,
        warn_bad_lines=True,
        delim_whitespace=False,
        low_memory=True,
        memory_map=False,
        float_precision=None,
    )

    dataset = dataset.shuffle(SHUFFLE_BUFFER_SIZE)
    dataset = dataset.batch(batch_size)

    return dataset


def run_fn(fn_args: FnArgs):
    """Train the model based on given args."""

    # Load the transformed data
    train_dataset = input_fn(
        fn_args.train_files,
        fn_args.transformed_metadata,
        batch_size=BATCH_SIZE
    )
    eval_dataset = input_fn(
        fn_args.eval_files,
        fn_args.transformed_metadata,
        batch_size=BATCH_SIZE
    )

    # Build the model
    model = build_model()

    # Train the model
    model.fit(
        train_dataset,
        epochs=fn_args.custom_config['train_epochs'],
        validation_data=eval_dataset,
        steps_per_epoch=fn_args.custom_config['steps_per_epoch'],
    )

    # Save the trained model
    model.save(fn_args.serving_model_dir, save_format='tf')


Overwriting cats_trainer.py


In [ ]:

# Train the model using Trainer
module_file = 'cats_trainer.py'
transform_module_file = 'cats_transform.py'

# Add Transform component
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=transform_module_file
)
context.run(transform)


trainer = Trainer(
    module_file=module_file,
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=10000),
    eval_args=trainer_pb2.EvalArgs(num_steps=5000)
)
context.run(trainer)

# Update Trainer component to use the transformed examples
trainer = Trainer(
    module_file=module_file,
    examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=10000),
    eval_args=trainer_pb2.EvalArgs(num_steps=5000)
)



# Evaluate the trained model using Evaluator
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model']
)
context.run(evaluator)


# Push the trained model using Pusher
push_destination = pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory=os.path.join(pipeline_root, 'serving_model')))

pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=push_destination
)
context.run(pusher)